In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 1000)
from utils import get_engine

engine = get_engine()

## Import and filter data

In [ ]:
delivery_smm = pd.read_csv("../data/processed/delivery_smm.csv",dtype={
    'record_id':str
})

# filter out 2016 data
delivery_smm = delivery_smm[
    ~delivery_smm.year.eq(2016)
]

# apply age/gender filter
delivery_smm = delivery_smm[
    delivery_smm.age_gender_filter.eq(1)
]

len(delivery_smm)

## Analyze

Summarize annual hospitalizations:

In [ ]:
delivery_smm.year.value_counts(sort=False)

Count of rows missing gestational weeks:

Rows missing gestational week:

In [ ]:
no_gw_len = len(delivery_smm[delivery_smm.weeks_gestation.isna()])

no_gw_len

In [ ]:
no_gw_len/len(delivery_smm)

#### Narrow to 13-21 weeks and create time bins

In [ ]:
early_records = delivery_smm.copy()[
    delivery_smm.weeks_gestation.lt(22)&
    delivery_smm.weeks_gestation.ge(13)
]

# create categories for nine quarters before and after SB 8
early_records.loc[
    early_records.year.ge(2022)|early_records.discharge.eq('2021Q4'),'time_bin'
] = 'after_sb8'
early_records.loc[
    early_records.discharge.isin(['2019Q3','2019Q4'])|early_records.year.eq(2020)|early_records.discharge.isin(['2021Q3','2021Q2','2021Q1']),'time_bin'
] = 'before_sb8'

Number of week 13-21 hospitalizations

In [ ]:
len(early_records)

In [ ]:
early_records.year.value_counts(sort=False)

Before/after summary table

In [ ]:
sepsis_summary = early_records.groupby('time_bin')[['pregnancy_end','sepsis_combined']].sum().reset_index().sort_values('time_bin',ascending=False)
sepsis_summary['pct_sepsis'] = sepsis_summary.sepsis_combined/sepsis_summary.pregnancy_end

sepsis_summary

#### Fetal demise

Hospitalizations with a diagnosis of fetal demise on admission

In [ ]:
no_hb_summary = early_records[early_records.no_fetal_heartbeat_admission.eq(1)].groupby('time_bin')[['pregnancy_end','sepsis_combined']].sum().reset_index().sort_values('time_bin',ascending=False)
no_hb_summary['pct_sepsis'] = no_hb_summary.sepsis_combined/no_hb_summary.pregnancy_end

no_hb_summary

Hospitalizations with no diagnosis of fetal demise on admission

In [ ]:
hb_summary = early_records[early_records.no_fetal_heartbeat_admission.eq(0)].groupby('time_bin')[['pregnancy_end','sepsis_combined']].sum().reset_index().sort_values('time_bin',ascending=False)
hb_summary['pct_sepsis'] = hb_summary.sepsis_combined/hb_summary.pregnancy_end

hb_summary